<a href="https://colab.research.google.com/github/GenaroHacker/creating_chord_collection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# @title Chord
class GuitarChord:
    def __init__(self, root, chord_type, starting_fret, difficulty, finger_ascending):
        self.root = root
        self.chord_type = chord_type
        self.starting_fret = starting_fret
        self.difficulty = difficulty
        self.finger_ascending = finger_ascending

    def __str__(self):
        return f"Guitar Chord: {self.root}{self.chord_type}, Starting Fret: {self.starting_fret}, Difficulty: {self.difficulty}, Finger Positions: {self.finger_ascending}"

    def get_string_notes(self):
        open_string_notes = ["E", "B", "G", "D", "A", "E"]
        all_notes = ["A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#"]

        # Default to 0 if starting_fret is None
        starting_fret = self.starting_fret if self.starting_fret is not None else 0

        string_notes = []

        for string, finger_position in zip(open_string_notes, self.finger_ascending):
            if finger_position is not None:
                # Calculate the note index, taking into account the starting fret
                note_index = (all_notes.index(string) + finger_position + starting_fret) % len(all_notes)
                string_notes.append(all_notes[note_index])
            else:
                # Append None for strings not played
                string_notes.append(None)

        return string_notes

    def is_open(self):
        return 0 in self.finger_ascending


# Example usage
chord = GuitarChord('F#', '+', 0, 3, [8, 1, 5, -2, None, None])
print(chord)
print("String Notes:", chord.get_string_notes())
print("Is Open Chord:", chord.is_open())



Guitar Chord: F#+, Starting Fret: 0, Difficulty: 3, Finger Positions: [8, 1, 5, -2, None, None]
String Notes: ['C', 'C', 'C', 'C', None, None]
Is Open Chord: False


In [14]:
# @title Collection
import sqlite3
from collections import defaultdict

class ChordCollection:
    def __init__(self):
        self.chords = defaultdict(list)

    def load(self, db_path):
        self.chords.clear()
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()

        cursor.execute('SELECT ROOT, TYPE, DIFFICULTY, STARTING_FRET, STRING_1, STRING_2, STRING_3, STRING_4, STRING_5, STRING_6 FROM TABLE_CHORDS')
        rows = cursor.fetchall()

        for row in rows:
            root, chord_type, difficulty, starting_fret, *fingers = row
            chord = GuitarChord(root, chord_type, starting_fret, difficulty, fingers)
            self.chords[root].append(chord)

        connection.close()

# Example usage
chord_collection = ChordCollection()
chord_collection.load('/content/chord_collection.db')
chord = chord_collection.chords["G"][0]
print(chord)
print(chord.get_string_notes())
print(chord.is_open())

Guitar Chord: G, Starting Fret: None, Difficulty: 3, Finger Positions: [3, 0, 0, 0, 2, 3]
['G', 'B', 'G', 'D', 'B', 'G']
True
